In [168]:
pip install cvxpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [169]:
# Install the needed libraries
import pandas as pd
import numpy as np
import time
import warnings
from pandas.core.common import SettingWithCopyWarning
import yaml
import cvxpy as cp
from datetime import datetime

In [170]:
# Ignore SettingWithCopyWarning 
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

In [171]:
def load_config():
    try:
        with open('config_file.yaml') as f:
            global constants
            constants = yaml.safe_load(f)
    except OSError as e:
        raise e
    except Exception as e:
        raise e

In [172]:
load_config()

In [173]:
data = pd.read_excel(constants['INPUT_FILE_NAME'], sheet_name='data', index_col=0)
real = pd.read_excel(constants['INPUT_FILE_NAME'], sheet_name='реал', index_col=0)
bandwidth = pd.read_excel(constants['INPUT_FILE_NAME'], sheet_name='ПС', index_col=0)
match = pd.read_excel(constants['INPUT_FILE_NAME'], sheet_name='match', index_col=0)

In [174]:
# Create a dict of names on a spreadsheets that we are going to use
names = {"Тариф ж/д" : "t1",
        "Тариф хранение" : "t2",
        "Тариф бренд" : "t3",
        "Тариф ВЛ" : "t4",
        "Плечо, км" : "distance",
        "Дата" : "date",
        "НБ" : "origin",
        "ОУ" : "point",
        "НГ" : "brand",
        "НГ_ПС" : "product",
        "Объем" : "volume"} 

In [175]:
# Rename the names in dataframes
def rename_columns(df_list, names):
    for df in df_list:
        df.rename(columns=names, inplace=True)

# Add the column with the tariff
def add_tariff(df):
    df['cost'] = df['t1'] + df['t2'] + df['t3'] + df['t4']
    df.loc[df['distance'] > 50, 'cost'] = df["t1"] + df["t2"] + df["t3"] +  df["t4"] * df["distance"]

# Make subset for the concrete time period
def select_time_period(df):
    df_subset = df[df['date'] == datetime.strptime(constants['MONTH'], '%Y-%m-%d')]
    return df_subset

# Adjust datatypes
def adjust_datatypes(df_list):
    for df in df_list:
        if 'origin' in df.columns:
            df['origin'] = df['origin'].map(lambda x: int(x.strip('Нефтебаза ')))
        if 'point' in df.columns:
            df['point'] = df['point'].map(lambda x: int(x.strip('АЗС ')))

In [176]:
# Apply changes to dataframes
def apply_changes_dfs(data, real, bandwidth, match, names):
    rename_columns([data, real, bandwidth, match], names)
    add_tariff(data)
    
    data_subset = select_time_period(data)
    real_subset = select_time_period(real)
    bandwidth_subset = select_time_period(bandwidth)

    adjust_datatypes([data_subset,real_subset, bandwidth_subset])
    
    return data_subset, real_subset, bandwidth_subset

In [177]:
data_subset, real_subset, bandwidth_subset = apply_changes_dfs(data, real, bandwidth, match, names)

In [178]:
# Create brand-product dict
def create_brand_product_dict(df):
    match_brand_product = dict(zip(df['brand'], df['product']))
    
    match_product_brand = dict()
    for key, value in match_brand_product.items():
        match_product_brand.setdefault(value, list()).append(key)
    return match_brand_product, match_product_brand

# Create numeration dict and inverted dict
def create_numeration_dict(df, parameter):
    match_dict = dict(enumerate(df[parameter]))
    inv_dict = {v: k for k, v in match_dict.items()}
    return match_dict, inv_dict

In [179]:
# Create 6 dictionaries (brand-product, brand-number, product-number, and inverted)
match_brand_product, match_product_brand = create_brand_product_dict(match)
match_brand_number, match_number_brand = create_numeration_dict(match,'brand')
match_product_number, match_number_product = create_numeration_dict(match,'product')

In [180]:
# Create name of variable
def create_var_Name(row):
    var_name = 'P' + str(int(row["point"])) + 'O' + str(int(row["origin"])) +  '_brand_' + str(int(row["brand_number"]))
    return var_name

# Reorganize the data
def reorganize_df(df, columns_to_keep, match_number_brand, match_number_product):
    df_reorganized = df.reset_index().loc[:, columns_to_keep]
    df_reorganized['brand_number'] = df_reorganized['brand'].map(lambda x: match_number_brand.get(x))
    df_reorganized['product_number'] = df_reorganized['product'].map(lambda x: match_number_product.get(x))
    
    return df_reorganized

In [181]:
columns_to_keep = ['origin', 'point', 'brand', 'product', 'cost']
df_reorganized = reorganize_df(data_subset, columns_to_keep, match_number_brand, match_number_product)

# Create a name variable
df_reorganized['var_Name'] = df_reorganized.apply(lambda row: create_var_Name(row), axis=1) 

In [182]:
# Create demand df
def create_demand_supply_dfs(real, bandwidth):
    demand = real[['point', 'brand', 'product', 'volume']]
    demand.rename(columns={'volume':'demand'}, inplace=True)
    demand['brand_number'] = demand['brand'].map(lambda x: match_number_brand.get(x))
    demand['product_number'] = demand['product'].map(lambda x: match_number_product.get(x))
    
    supply = bandwidth[['origin', 'product', 'volume']]
    supply.rename(columns={'volume':'supply'}, inplace=True)
    supply['product_number'] = supply['product'].map(lambda x: match_number_product.get(x))
    
    return demand, supply

In [183]:
demand, supply = create_demand_supply_dfs(real_subset, bandwidth_subset)

# CVXPY

In [184]:
brands = df_reorganized['brand_number'].unique()

In [185]:
import json

df_reorganized

brand_info_map = {}

def make_brand(brand_id, name):
    return {'name': name, 'id': brand_id, 'product_transportations': {}}

def make_transportation(origin, point, cost, var_name):
    return {'origin': origin, 'point': point, 'cost': cost, 'var_name': var_name}

def add_transportation_to_brand(brand, product_id, transportation):
    if brand['product_transportations'].get(product_id) == None:
        brand['product_transportations'][product_id] = [transportation];
        return brand;
    
    brand['product_transportations'].get(product_id).append(transportation);    

    return brand;


def append_row_to_map(brand_map, row, index):
    brand_id = row['brand_number'];
    
    if brand_map.get(brand_id) == None:
        brand_map[brand_id] = make_brand(brand_id, row['brand']);
        
    brand_item = brand_map.get(brand_id);
    product_id = row['product_number'];
    
    transportation = make_transportation(row['origin'], row['point'], row['cost'], index);
    brand_map[brand_id] = add_transportation_to_brand(brand_item, product_id, transportation)
    
    return brand_map

for i, row in df_reorganized.iterrows():
    brand_info_map = append_row_to_map(brand_info_map, row, i);

json_formatted_str = json.dumps(brand_info_map, indent=2, ensure_ascii=False)

In [186]:
# Function for dividing supply between different brands
def merge_supply(brand_number, match_brand_product, match_brand_number, match_product_brand, match_number_brand):
    """This function returns a set of brands' numbers that are produced from the same product"""
    brand_name = match_brand_number[brand_number]
    product_name = match_brand_product[brand_name]
    brands_overlap_names = match_product_brand[product_name]
    brands_overlap_numbers = set(match_number_brand.get(item) for item in brands_overlap_names)

    return brands_overlap_numbers

In [187]:
def define_problem(df):
    x = cp.Variable(len(df), nonneg = True)
    c = np.array(df['cost'])
    objective = cp.Minimize(c.T @ x)
    
    return x, objective

In [188]:
def extract_unique_points_of_brand(brand):
    points = []
    for product_id in brand.get('product_transportations').keys():
        for transportation in brand.get('product_transportations').get(product_id):
            points.append(transportation.get('point'))

    points = list(set(points))
    
    return points

def extract_unique_origins_of_brand(brand):
    origins = []
    for transportations in brand.get('product_transportations').values():
        for transportation in transportations:
            origins.append(transportation.get('origin'))

    origins = list(set(origins))
    
    return origins

def get_brand_transportations(brand):
    brand_transpotrations = []
    for product_id in brand.get('product_transportations').keys():
        for transportation in brand.get('product_transportations').get(product_id):
            brand_transpotrations.append(transportation)
                
    return brand_transpotrations

def extract_unique_origins(brand_map):
    origins = []
    for brand in brand_map.values():
        origins = origins + extract_unique_origins_of_brand(brand)

    return list(set(origins))

def extract_unique_points(brand_map):
    points = []
    for brand in brand_map.values():
        points = points + extract_unique_points_of_brand(brand)
        
    return list(set(points))

def get_transportations(brand_map):
    transportations = []
    
    for brand in brand_map.values():
        transportations = transportations + get_brand_transportations(brand)
        
    return transportations

#### Define restriction #1: demand equals const

In [189]:
def make_restriction_demand(record, brands, x, demand):
    n_constraint = 1
    constraints = []
    
    for brand_id in brands:
        brand = record.get(brand_id);
        points = extract_unique_points_of_brand(brand)
    
        for p in points:
            transportations = get_brand_transportations(brand)
            brand_point_transpotrations = [transportation for transportation in transportations if transportation.get('point') == p]
            
            const = float(demand[(demand['point'] == p) & (demand['brand_number'] == brand_id)]['demand'])
            transportations_x = [x[transportation.get('var_name')] for transportation in brand_point_transpotrations]
            constraints.append(cp.sum(transportations_x) == const)
            n_constraint +=1
        
    return constraints, n_constraint, x

#### Define restriction #2: supply less or equal than const;  restriction #4: supply greater or equal to const * percent_loading

In [190]:
def make_restriction_supply(constraints, record, bandwidth, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint):
    # Set for storing already processed brands 
    brands_accounted_for = set()
    
    # Store origin with infinite supply in a separate list
    origins_with_inf_supply = list(bandwidth[bandwidth['volume'] == np.inf]['origin'].unique())
    
    for b in brands:
        if b in brands_accounted_for:
            continue
        
        brands_overlap_numbers = merge_supply(b, match_brand_product, match_brand_number, match_product_brand, match_number_brand)
        brands_accounted_for.update(brands_overlap_numbers)

        brands_list = [brand for brand in record.values() if brand.get('id') in brands_overlap_numbers]

        origins = []
        for brand in brands_list:
            origins += extract_unique_origins_of_brand(brand)
            
        origins = list(set(origins))
        
        transportations_list = []
        for brand in brands_list:
            transportations_list += get_brand_transportations(brand)

        for o in origins:
            if o in origins_with_inf_supply:
                continue

            origin_transportations = [transportation for transportation in transportations_list if transportation.get('origin') == o]

            product_name = match_brand_product[match_brand_number[b]]
            const = float(supply[(supply['origin'] == o) & (supply['product'] == product_name)]['supply'])

            # Add a restriction on supply <= upper_const
            transportation_x = [x[transportation.get("var_name")] for transportation in origin_transportations]
            constraints.append(cp.sum(transportation_x) <= const)
            n_constraint +=1

            # Add a restriction on supply >= lower_const
            const *= constants['PERCENT_LOADING']
            transportation_x = [x[transportation.get("var_name")] for transportation in origin_transportations]
            constraints.append(cp.sum(transportation_x) >= const)
            n_constraint +=1 
                    
    return constraints, n_constraint, x

#### Define restriction #3: all brands for one point should come from the same origin

In [191]:
def create_dummy_Name(row):
    return 'y_P' + str(int(row["point"])) + 'O' + str(int(row["origin"])), int(row["point"]), int(row["origin"])

In [192]:
def make_restriction_dummy(df, x, constraints, n_constraint):
    [*o] = df.apply(lambda row: create_dummy_Name(row), axis=1)
    df_dummies = pd.DataFrame(o, columns=['dummy_Name', 'point', 'origin'])
    df_dummies = df_dummies.reset_index()
    df_dummies = df_dummies.drop_duplicates(subset=['dummy_Name', 'point', 'origin'])
    dummies = cp.Variable(len(df_dummies), boolean=True)
    
    df_dummies.reset_index(inplace=True)
    df_dummies.drop(columns=['index', 'level_0'], inplace=True)
    
    points = df['point'].unique()
    origins = df['origin'].unique()

    for p in points:
        for o in origins:
            df1 = df[(df['point'] == p) & (df['origin'] == o)]
            df_d = df_dummies[(df_dummies['point'] == p) & (df_dummies['origin'] == o)]
            constraints.append(cp.sum(x[df1.index]) <= constants['M'] * dummies[df_d.index])
            n_constraint += 1
    
    const = 1     
    for p in points:
        df_d = df_dummies[df_dummies['point'] == p]
        constraints.append(cp.sum(dummies[df_d.index]) == const)
        n_constraint += 1


    return constraints

In [193]:
x, objective = define_problem(df_reorganized)
constraints, n_constraint, x = make_restriction_demand(brand_info_map, brands, x, demand)
constraints, n_constraint, x = make_restriction_supply(constraints, brand_info_map, bandwidth_subset, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint)
constraints = make_restriction_dummy(df_reorganized, x, constraints, n_constraint)

In [194]:
def main(objective, constraints):
    problem = cp.Problem(objective, constraints)
    problem.solve(verbose=True)

In [196]:
if __name__ == '__main__':
    main(objective, constraints)

                                     CVXPY                                     
                                     v1.3.1                                    
(CVXPY) Apr 23 10:20:19 PM: Your problem has 458 variables, 335 constraints, and 0 parameters.
(CVXPY) Apr 23 10:20:19 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 23 10:20:19 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 23 10:20:19 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 23 10:20:19 PM: Compiling problem (target solver=COPT).
(CVXPY) Apr 23 10:20:19 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing